<section style="position:relative; width:100%;">

<div style="display:inline-block; float:right">
    <h3>Lycée Marie curie</h3>
    <p>Terminale NSI</p>
</div>

</section>


<h1 style="text-align: center; width:100%">Alignement de séquences</h1>

# 1. L'alignement de séquences en bio-informatique

Dans le domaine de la bio-informatique, un problème algorithmique essentiel est celui de l’alignement de séquences, typiquement pour comparer des séquences d'ADN. On peut le formaliser en ces termes :

- Etant données deux chaı̂nes de caractères, comment mettre _le plus possible_ en correspondance les caractères de la première chaı̂ne avec ceux de la seconde chaı̂ne, en conservant l'ordre des caractères mais en s’autorisant à insérer des trous dans l'une ou l’autre des chaı̂nes.

- La seule contrainte est que l’on doit utiliser tous les caractères de chaque chaı̂ne, dans l’ordre, et que l’on n’aligne jamais deux trous.

- Parmi tous les alignements possibles, on cherche celui qui donne le score maximal : on ajoute un point quand deux caractères identiques sont alignés, mais on soustrait un point quand ce n'est pas le cas (un trou est un caractère).

### Exemple :
Si on considère les chaı̂nes `GENOME` et `ENORME`, en utilisant le symbole `-` pour désigner un trou, on peut
aligner leurs caractères de la façon suivante :
```
GENO-ME                  G--ENO-ME
-ENORME                  -EN-O-RME

score: 5-2=3 (max)       score: 2-7=-5
``` 

### Exercice :
Déterminer le score max pour l'alignement des mots `CHAT` et `CAT`.

# 2. Solution récurssive

Reprenons l'exemple précédent et considéront la dernière lettre de chaque mot ainsi que le caractère `-`.

Il y a trois possibilité :
```
ENORM, E          ENORM, E         ENORME, -
GENOM, E         GENOME, -          GENOM, E
```

- `E` et `E` sont alignés, le score est alors égal au score de l'alignement de `GENOM` et `ENORM` plus 1 car les dernières lettres de chaque mot sont les mêmes,
- `E` et `-` sont alignés, le score est alors égal au score de l'alignement de `GENOM` et `ENORME` moins 1,
- `-` et `E` sont alignés, le score est alors égal au score de l'alignement de `GENOME` et `ENORM` moins 1.

Le score maximal est alors le maximum de ces trois possibilités. Chaque possibilité est calculer récursivement avec le même procédé.

De manière général, si l'on considère deux mots $a=a_0a_1...a_n$ et $b = b_0 b_1 ... b_m$, tels que $a$ soit traité jusqu'a $a_i$ et $b$ jusqu'a $b_j$, il y a trois possibilités :
- on aligne $a_{i-1}$ avec $b_{j-1}$
- on aligne $a_{i-1}$ avec $-$
- on aligne $b_{j-1}$ avec $-$

Ce qui donne la formule de récurrence suivante :
$$
score(a,i,b,j) = \max\left\{
    \begin{array}\\
        score(a,i-1,b,j-1)+1 & \mbox{si } \ a_{i-1} = b_{j-1} \\
        score(a,i-1,b,j-1)-1 & \mbox{si } \ a_{i-1} \neq b_{j-1} \\
        score(a,i,b,j-1)-1 \\
        score(a,i-1,b,j)-1
    \end{array}
\right.
$$

Le processus ce termine lorsque l'un des deux mots n'a plus de caractère, dans ce cas le score est négatif et correspond aux nombres de lettres restantes.

### Exercice
Compléter le programme récursif suivant afin qu'il réalise cet algorithme.

In [61]:
def score_rec(a, i, b, j):
    #s'il n'y a plus de lettre dans le mot a
    if i == 0:
        return ???
    #s'il n'y a plus de lettre dans le mot b
    elif j == 0:
        return ???
    else:
        #si a[i-1] et b[i-1] sont alignés
        if a[i-1] == b[j-1]:
            s1 = ???
        else:
            s1 = ???
        #si a[i-1] et - sont alignés
        s2 = ???
        #si b[i-1] et - sont alignés
        s3 = ???
        return max(s1, s2, s3)

In [62]:
a, b = 'ENORME', 'GENOME'
n, m = len(a), len(b)
score_rec(a, n, b, m)

3

In [63]:
a, b = 'CHAT', 'CAT'
n, m = len(a), len(b)
score_rec(a, n, b, m)

2

# 3. Programmation dynamique

Comme dans le rendu de monnaie, on peut améliorer notre programme en évitant des appels redondants grâce à la programmation dynamique.

Nous allons utiliser un tableau à deux dimensions dans lequel on va stocker, pour chaque pair de mots, le score maximal. Plus précisément, pour deux entiers $i$ et $j$, on va stocker dans la case $(i,j)$ de ce tableau le score maximal de l'alignement des $i$ premiers caractères du premier mot et des $j$ premiers caractères du deuxième mot.

### Exemple
Pour les mots `GENOME` et `ENORME`, on obtient :

```
        E   N   O   R   M   E
  [ 0, -1, -2, -3, -4, -5, -6]
G [-1, -1,  0, -1, -2, -3, -4]
E [-2, -2, -1,  1,  0, -1, -2]
N [-3, -3, -2,  0,  2,  1,  0]
O [-4, -4, -3, -1,  1,  1,  0]
M [-5, -5, -4, -2,  0,  2,  1]
E [-6, -6, -4, -3, -1,  1,  3]

```

La première ligne et la première colonne correspondent au cas d'arrêt de l'algorithme récursif, c'est à dire au cas où un mot à été complètement traité.

Le reste est obtenu à l'ai de la formule précédente :

$$
score[i][j] = \max\left\{
    \begin{array}\\
        score[i-1][j-1]+1 & \mbox{si } \ a_{i-1} = b_{j-1} \\
        score[i-1][j-1]-1 & \mbox{si } \ a_{i-1} \neq b_{j-1} \\
        score[i,j-1]-1 \\
        score[i-1,j]-1
    \end{array}
\right.
$$

### Exercices
1. Calculer à la main le tableau des scores d'alignement pour les deux mots `CHAT` et `CAT`.
2. Compléter le programme ci-dessous afin qu'il réalise l'algorithme décrit.

In [67]:
def score_dyn(a, b):
    n, m = len(a), len(b)
    score = [[0 for _ in range(m+1)] for _ in range(n+1)]
    #première ligne
    for i in range(n+1):
        ???
    #première colonne
    for j in range(m+1):
        ???
    # le reste
    for i in range(1,n+1):
        for j in range(1, m+1):
            if a[i-1] == b[j-1]:
                s1 = ???
            else:
                s1 = ???
            s2 = ???
            s3 = ???
            score[i][j] = max(s1, s2, s3)
    return ???

In [68]:
score_dyn('ENORME','GENOME')

3

In [69]:
score_dyn('CAT','CHAT')

2

3 Ecrire une fonction `affiche(a, b, score)` qui affiche la table `score` calculer dans la fonction `score_dyn` sou la forme suivante :
```
      E  N  O  R  M  E
   0 -1 -2 -3 -4 -5 -6
G -1 -1  0 -1 -2 -3 -4
E -2 -2 -1  1  0 -1 -2
N -3 -3 -2  0  2  1  0
O -4 -4 -3 -1  1  1  0
M -5 -5 -4 -2  0  2  1
E -6 -6 -4 -3 -1  1  3
```
4 Modifier la fonction `score_dyn` sous la forme d'un couple de chaînes de caractères contenant d'éventuels caractères `-`. 

In [ ]:
def score_dyn(a, b):
    n, m = len(a), len(b)
    score = [[0 for _ in range(m+1)] for _ in range(n+1)]
    ...

5 Dans le problème de l'alignement de séquences d'ADN, on utilise en pratique un calcul des scores plus subtil, qui dépend des caractères qui sont alignés. Pour cela, on se donne une _matrice de similarités_. Par exemple, pour les chaînes formées uniquement des caractères `A`, `G`, `C` et `T`, on peut se donner la matrice :
```
    A  G  C  T
A  10 -1 -3 -4
G  -1  7 -5 -3
C  -3 -5  9  0
T  -4 -3  0  8
```
Par ailler on se donne une variable `gap` pour le score d'un alignement avec le caractère `-` (par exemple `gap=-5`).

Modifier la fonction `score_dyn` pour qu'elle utilise ces deux variables dans le calcul du score.

In [ ]:
gap = -5
sim = {
    'A':{A:10}, {G:-1}, {C:-3}, {T:-4},
    ...
}

def score_dyn(a,b):
    ...